In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from keras.models import Sequential
from keras.layers import LSTM
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.ensemble import RandomForestClassifier


from imblearn.under_sampling import (ClusterCentroids, RandomUnderSampler,
                                     NearMiss,
                                     InstanceHardnessThreshold,
                                     CondensedNearestNeighbour,
                                     EditedNearestNeighbours,
                                     RepeatedEditedNearestNeighbours,
                                     AllKNN,
                                     NeighbourhoodCleaningRule,
                                     OneSidedSelection)
from sklearn.svm import LinearSVC
from imblearn.pipeline import make_pipeline


Using TensorFlow backend.


In [2]:
dataset = pd.read_csv('../../output/df_general.csv', sep=',')
split = 0.8
del dataset['No.']
del dataset['Protocol']
del dataset['tcp_flag_cwr']
del dataset['tcp_flag_ecn']
del dataset['tcp_flag_urg']
del dataset['ip_len']
del dataset['tcp_flag_psh']
del dataset['tcp_flag_fin']
del dataset['prebuffering']
del dataset['Time']

print(dataset.columns)
print(Counter(dataset['stalling_event']))

Index(['∆t', 'pkt_len', 'tcp_hea_len', 'tcp_flag_ack', 'tcp_flag_rst',
       'tcp_flag_syn', 'tcp_win_si2', 'delay', 'capture', 'stalling_event'],
      dtype='object')
Counter({0: 1426905, 1: 113031})


In [3]:
from random import randint
'''X_train = [None] * (len(dataset.columns)-1)
X_test = [None] * (len(dataset.columns)-1)'''

X_train = np.array([None]*7)
X_test = np.array([None]*7)
y_train = list()
y_test = list()

grouped = dataset.groupby(['delay','capture']) 
#print(grouped.mean())

for id, data in grouped:
    spl = int(data.shape[0] * split);
    restspl = int(data.shape[0] * (1-split))
    randnum = randint(0,restspl)
    print('data.shape: '+str(data.shape)+' | spl: '+str(spl)+
          ' | restspl: '+str(restspl)+' | randnum: '+str(randnum)+
          ' | 0:randnum='+str(range(0,randnum)) +' | randnum:randnum+spl= '+str(range(randnum,randnum+spl))+
          ' | randnum+spl: = '+str(range(randnum+spl,data.shape[0])))
    X_train = np.vstack((X_train,data.values[randnum:randnum+spl,:-3]))
    X_test = np.vstack((X_test,data.values[0:randnum-1,:-3]))
    X_test = np.vstack((X_test,data.values[randnum+spl:,:-3]))
    
    y_train.extend(data.values[randnum:randnum+spl,-1])
    y_test.extend(data.values[0:randnum-1,-1])
    y_test.extend(data.values[randnum+spl:,-1])
    '''X_train.extend(data.loc[randnum:randnum+spl][data.columns[:-3]].values.tolist())
    X_test.extend(data.loc[0:randnum][data.columns[:-3]].values.tolist())
    X_test.extend(data.loc[randnum+spl:][data.columns[:-3]].values.tolist())
    y_train.extend(data.loc[randnum:randnum+spl,data.columns=='stalling_event'].values.tolist())
    y_test.extend(data.loc[0:randnum,data.columns=='stalling_event'].values.tolist())
    y_test.extend(data.loc[randnum+spl:,data.columns=='stalling_event'].values.tolist())'''
X_train = X_train[1:]
X_test = X_test[1:]
print(len(y_train))
print(X_train.shape)
print(len(y_test))
print(X_test.shape)
#print('X_train: '+str(len(X_train))+' | X_test: '+str(len(X_test)) + ' | y_train: '+str(len(y_train)) + '| y_test: '+str(len(y_test)))
#print(dataset.columns[:-3])
#print(X_train)


data.shape: (25242, 10) | spl: 20193 | restspl: 5048 | randnum: 4477 | 0:randnum=range(0, 4477) | randnum:randnum+spl= range(4477, 24670) | randnum+spl: = range(24670, 25242)
data.shape: (26007, 10) | spl: 20805 | restspl: 5201 | randnum: 2063 | 0:randnum=range(0, 2063) | randnum:randnum+spl= range(2063, 22868) | randnum+spl: = range(22868, 26007)
data.shape: (26266, 10) | spl: 21012 | restspl: 5253 | randnum: 4257 | 0:randnum=range(0, 4257) | randnum:randnum+spl= range(4257, 25269) | randnum+spl: = range(25269, 26266)
data.shape: (26339, 10) | spl: 21071 | restspl: 5267 | randnum: 2831 | 0:randnum=range(0, 2831) | randnum:randnum+spl= range(2831, 23902) | randnum+spl: = range(23902, 26339)
data.shape: (30713, 10) | spl: 24570 | restspl: 6142 | randnum: 738 | 0:randnum=range(0, 738) | randnum:randnum+spl= range(738, 25308) | randnum+spl: = range(25308, 30713)
data.shape: (24957, 10) | spl: 19965 | restspl: 4991 | randnum: 1565 | 0:randnum=range(0, 1565) | randnum:randnum+spl= range(156

In [4]:
y_test = list(map(int,y_test))
y_train = list(map(int,y_train))
print(y_test)
print(Counter(y_test))
print(Counter(y_train))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [5]:
X_train = pd.DataFrame(X_train)
X_test= pd.DataFrame(X_test)

X_test = X_test.fillna(X_test.mean())
X_train = X_train.fillna(X_train.mean())

y_train = np.array(y_train)
y_test  = np.array(y_test)

In [6]:
norm = preprocessing.MinMaxScaler()
X_train_norm = norm.fit_transform(X_train)
X_test_norm = norm.transform(X_test)
print(X_test_norm)
print(Counter(y_train))
print(Counter(y_test))

[[0.00000000e+00 1.63043478e-02 3.33333333e-01 ... 0.00000000e+00
  1.00000000e+00 1.25384557e-01]
 [6.93516574e-03 1.63043478e-02 3.33333333e-01 ... 0.00000000e+00
  1.00000000e+00 9.94872580e-01]
 [4.24169159e-05 8.15217391e-03 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 3.91826739e-03]
 ...
 [1.55614939e+00 1.22282609e-02 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 9.83179001e-01]
 [6.36253738e-05 8.15217391e-03 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 9.66067192e-01]
 [3.19696295e-01 1.22282609e-02 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 9.83163695e-01]]
Counter({0: 1122787, 1: 109137})
Counter({0: 304058, 1: 3894})


In [7]:
std = preprocessing.StandardScaler()
X_train_std = std.fit_transform(X_train)
X_test_std = std.transform(X_test)
print(X_test_std)

[[-1.32487638e-01 -1.24600932e+00  1.98446328e+00 ... -5.47491180e-02
   2.66509368e+01 -1.25814209e+00]
 [ 5.38714053e-01 -1.24600932e+00  1.98446328e+00 ... -5.47491180e-02
   2.66509368e+01  6.86585396e-01]
 [-1.28382429e-01 -1.26317141e+00 -4.05864284e-01 ... -5.47491180e-02
  -3.75252706e-02 -1.52981786e+00]
 ...
 [ 1.50475319e+02 -1.25459036e+00 -4.05864284e-01 ... -5.47491180e-02
  -3.75252706e-02  6.60431125e-01]
 [-1.26329824e-01 -1.26317141e+00 -4.05864284e-01 ... -5.47491180e-02
  -3.75252706e-02  6.22158253e-01]
 [ 3.08084735e+01 -1.25459036e+00 -4.05864284e-01 ... -5.47491180e-02
  -3.75252706e-02  6.60396892e-01]]


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE

import sklearn.metrics as metrics

rs = 42

def compute_metrics (y_test2,y_pred):
    accuracy = metrics.accuracy_score(y_test2,y_pred)
    print("Accuracy: %.2f%%" % (accuracy * 100.0))
    
    precision = metrics.precision_score(y_test2,y_pred)
    print("Precision: %.2f%%" % (precision * 100.0))
    
    recall = metrics.recall_score(y_test2,y_pred)
    print("Recall: %.2f%%" % (recall * 100.0))
    
    F1Score = metrics.f1_score(y_test2,y_pred)
    print("F1 Score: %.2f%%" % (F1Score * 100.0))
    
    print('CONFUSION MATRIX:')
    conf_m = metrics.confusion_matrix(y_test2,y_pred)
    print(conf_m)
    print()

def plot_2d_space(X, y, label='Classes'):   
    colors = ['#1F77B4', '#FF7F0E']
    markers = ['o', 's']
    for l, c, m in zip(np.unique(y), colors, markers):
        plt.scatter(
            X[y==l, 0],
            X[y==l, 1],
            c=c, label=l, marker=m
        )
    plt.title(label)
    plt.legend(loc='upper right')
    plt.show()

def adaboost(X_train, X_test, y_train):
    model = AdaBoostClassifier(n_estimators=100,random_state=rs)
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    return y_pred

def RandomForest (X_train,y_train,X_test,y_test):
    for i in range(1,3):
        print('dep: '+str(i))
        rfc = RandomForestClassifier(max_depth=i, random_state=rs,verbose=True)
        rfc.fit(X_train,y_train)
        y_pred = rfc.predict(X_test)
        compute_metrics(y_test,y_pred)

In [9]:
from imblearn.over_sampling import SMOTE
print('Original X_train shape: '+str(X_train_std.shape)+' | Original y_train shape: '+str(len(y_train)))

y_baseline = adaboost(X_train_std,X_test_std,y_train)
print('BASELINE METRICS(ADABOOST)')
compute_metrics(np.array(y_test),y_baseline)
print('\nBASELINE METRICS(RF)')
y_bs = RandomForest(X_train_std,y_train,X_test_std,y_test)
print('\nBASELINE METRICS(LinearSVC)')
lsvc = LinearSVC(verbose=True,random_state=rs).fit(X_train_std,y_train)
y_bs2 = lsvc.predict(X_test_std)
compute_metrics(np.array(y_test),y_bs2)

Original X_train shape: (1231924, 7) | Original y_train shape: 1231924
BASELINE METRICS(ADABOOST)
Accuracy: 98.74%
Precision: 0.00%
Recall: 0.00%
F1 Score: 0.00%
CONFUSION MATRIX:
[[304058      0]
 [  3894      0]]


BASELINE METRICS(RF)
dep: 1
Accuracy: 98.74%
Precision: 0.00%
Recall: 0.00%
F1 Score: 0.00%
CONFUSION MATRIX:
[[304058      0]
 [  3894      0]]

dep: 2
Accuracy: 98.74%
Precision: 0.00%
Recall: 0.00%
F1 Score: 0.00%
CONFUSION MATRIX:
[[304058      0]
 [  3894      0]]


BASELINE METRICS(LinearSVC)
[LibLinear]Accuracy: 98.74%
Precision: 0.00%
Recall: 0.00%
F1 Score: 0.00%
CONFUSION MATRIX:
[[304058      0]
 [  3894      0]]



c:\users\usuario\pycharmprojects\stallingevents\venv\lib\site-packages\sklearn\metrics\_classification.py:1268: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\usuario\pycharmprojects\stallingevents\venv\lib\site-packages\sklearn\metrics\_classification.py:1268: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\usuario\pycharmprojects\stallingevents\venv\lib\site-packages\sklearn\metrics\_classification.py:1268: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\usuario\pycharmprojects\s

In [10]:
sm = SMOTE(random_state=rs)
X_sm, y_sm = sm.fit_resample(X_train_norm, y_train)
print('X_sm shape: '+str(X_sm.shape)+' | y_sm shape: '+str(y_sm.shape))
print(Counter(y_sm))
y_smote = adaboost(X_sm,X_test_norm,y_sm)
print('SMOTE - NORM | METRICS(ADABOOST)')
compute_metrics(np.array(y_test),y_smote)
print('\nSMOTE - NORM | METRICS(RF)')
RandomForest(X_sm,y_sm,X_test_norm,y_test)
print('\nSMOTE - NORM | METRICS(LinearSVC)')
lsvc = LinearSVC(verbose=True,random_state=rs).fit(X_train_norm,y_train)
y_bs2 = lsvc.predict(X_test_norm)
compute_metrics(np.array(y_test),y_bs2)

X_sm shape: (2245574, 7) | y_sm shape: (2245574,)
Counter({0: 1122787, 1: 1122787})
SMOTE - NORM | METRICS(ADABOOST)
Accuracy: 63.40%
Precision: 0.93%
Recall: 26.43%
F1 Score: 1.79%
CONFUSION MATRIX:
[[194200 109858]
 [  2865   1029]]


SMOTE - NORM | METRICS(RF)
dep: 1
Accuracy: 53.32%
Precision: 1.15%
Recall: 42.24%
F1 Score: 2.24%
CONFUSION MATRIX:
[[162559 141499]
 [  2249   1645]]

dep: 2
Accuracy: 57.00%
Precision: 1.07%
Recall: 36.03%
F1 Score: 2.08%
CONFUSION MATRIX:
[[174139 129919]
 [  2491   1403]]


SMOTE - NORM | METRICS(LinearSVC)
[LibLinear]Accuracy: 98.74%
Precision: 0.00%
Recall: 0.00%
F1 Score: 0.00%
CONFUSION MATRIX:
[[304058      0]
 [  3894      0]]



c:\users\usuario\pycharmprojects\stallingevents\venv\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
c:\users\usuario\pycharmprojects\stallingevents\venv\lib\site-packages\sklearn\metrics\_classification.py:1268: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [11]:
sm = SMOTE(random_state=rs)
X_sm, y_sm = sm.fit_resample(X_train_std, y_train)
print('X_sm shape: '+str(X_sm.shape)+' | y_sm shape: '+str(y_sm.shape))
print(Counter(y_sm))
y_smote = adaboost(X_sm,X_test_std,y_sm)
print('SMOTE - STD | METRICS(ADABOOST)')
compute_metrics(np.array(y_test),y_smote)
print('\nSMOTE - STD | METRICS(RF)')
RandomForest(X_sm,y_sm,X_test_std,y_test)
print('\nSMOTE - STD | METRICS(LinearSVC)')
lsvc = LinearSVC(verbose=True,random_state=rs).fit(X_train_std,y_train)
y_bs2 = lsvc.predict(X_test_std)
compute_metrics(np.array(y_test),y_bs2)

X_sm shape: (2245574, 7) | y_sm shape: (2245574,)
Counter({0: 1122787, 1: 1122787})
SMOTE - STD | METRICS(ADABOOST)
Accuracy: 59.39%
Precision: 0.74%
Recall: 23.47%
F1 Score: 1.44%
CONFUSION MATRIX:
[[181964 122094]
 [  2980    914]]


SMOTE - STD | METRICS(RF)
dep: 1
Accuracy: 53.32%
Precision: 1.15%
Recall: 42.24%
F1 Score: 2.24%
CONFUSION MATRIX:
[[162559 141499]
 [  2249   1645]]

dep: 2
Accuracy: 53.05%
Precision: 1.14%
Recall: 42.24%
F1 Score: 2.22%
CONFUSION MATRIX:
[[161718 142340]
 [  2249   1645]]


SMOTE - STD | METRICS(LinearSVC)
[LibLinear]Accuracy: 98.74%
Precision: 0.00%
Recall: 0.00%
F1 Score: 0.00%
CONFUSION MATRIX:
[[304058      0]
 [  3894      0]]



c:\users\usuario\pycharmprojects\stallingevents\venv\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
c:\users\usuario\pycharmprojects\stallingevents\venv\lib\site-packages\sklearn\metrics\_classification.py:1268: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
y_tra_df = pd.DataFrame(y_train,columns=['target'])
print(y_tra_df.shape)
print(X_train_norm.shape)
XT = pd.DataFrame(X_train_norm,columns=dataset.columns[:-3])
print(XT.head(10))
XT['target'] = y_tra_df 
# RUS
X_full = XT.copy()
X_maj = X_full[X_full.target==0]
X_min = X_full[X_full.target==1]
X_maj_rus = resample(X_maj,replace=False,n_samples=len(X_min),random_state=rs)
X_rus = pd.concat([X_maj_rus, X_min])
X_train_rus = X_rus.drop(['target'], axis=1)
y_train_rus = X_rus.target
print('X_rus shape: '+str(X_rus.shape)+' | X_train_rus shape: '+str(X_train_rus.shape)+' | y_train_rus shape: '+str(y_train_rus.shape))
print(Counter(y_train_rus))
y_rus = adaboost(X_train_rus, X_test_norm, y_train_rus)
print('RUS - NORM METRICS(ADABOOST)')
compute_metrics(np.array(y_test),y_rus)
print('RUS - NORM METRICS(RF)')
y_ru = RandomForest(X_train_rus,y_train_rus,X_test_norm,y_test)
print('\nRUS - NORM | METRICS(LinearSVC)')
lsvc = LinearSVC(verbose=True,random_state=rs).fit(X_train_norm,y_train)
y_bs2 = lsvc.predict(X_test_norm)
compute_metrics(np.array(y_test),y_bs2)


(1231924, 1)
(1231924, 7)
         ∆t   pkt_len  tcp_hea_len  tcp_flag_ack  tcp_flag_rst  tcp_flag_syn  \
0  0.000339  1.000000          0.0           1.0           0.0           0.0   
1  0.000021  0.008152          0.0           1.0           0.0           0.0   
2  0.000636  1.000000          0.0           1.0           0.0           0.0   
3  0.000318  1.000000          0.0           1.0           0.0           0.0   
4  0.000021  0.008152          0.0           1.0           0.0           0.0   
5  0.000000  1.000000          0.0           1.0           0.0           0.0   
6  0.004708  1.000000          0.0           1.0           0.0           0.0   
7  0.000064  0.008152          0.0           1.0           0.0           0.0   
8  0.000764  1.000000          0.0           1.0           0.0           0.0   
9  0.000170  1.000000          0.0           1.0           0.0           0.0   

   tcp_win_si2  
0     0.988582  
1     0.003918  
2     0.988582  
3     0.988582  
4     0.

c:\users\usuario\pycharmprojects\stallingevents\venv\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
c:\users\usuario\pycharmprojects\stallingevents\venv\lib\site-packages\sklearn\metrics\_classification.py:1268: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [15]:
y_tra_df = pd.DataFrame(y_train,columns=['target'])
print(y_tra_df.shape)
print(X_train_std.shape)
XT = pd.DataFrame(X_train_std,columns=dataset.columns[:-3])
print(XT.head(10))
XT['target'] = y_tra_df 
# RUS
X_full = XT.copy()
X_maj = X_full[X_full.target==0]
X_min = X_full[X_full.target==1]
X_maj_rus = resample(X_maj,replace=False,n_samples=len(X_min),random_state=rs)
X_rus = pd.concat([X_maj_rus, X_min])
X_train_rus = X_rus.drop(['target'], axis=1)
y_train_rus = X_rus.target
print('X_rus shape: '+str(X_rus.shape)+' | X_train_rus shape: '+str(X_train_rus.shape)+' | y_train_rus shape: '+str(y_train_rus.shape))
print(Counter(y_train_rus))
y_rus = adaboost(X_train_rus, X_test_std, y_train_rus)
print('RUS - STD METRICS(ADABOOST)')
compute_metrics(np.array(y_test),y_rus)
print('RUS - STD METRICS(RF)')
y_ru = RandomForest(X_train_rus,y_train_rus,X_test_std,y_test)
print('\nRUS - STD | METRICS(LinearSVC)')
lsvc = LinearSVC(verbose=True,random_state=rs).fit(X_train_norm,y_train)
y_bs2 = lsvc.predict(X_test_std)
compute_metrics(np.array(y_test),y_bs2)

(1231924, 1)
(1231924, 7)
         ∆t   pkt_len  tcp_hea_len  tcp_flag_ack  tcp_flag_rst  tcp_flag_syn  \
0 -0.099646  0.824884    -0.405864      0.054876     -0.054749     -0.037525   
1 -0.130435 -1.263171    -0.405864      0.054876     -0.054749     -0.037525   
2 -0.070910  0.824884    -0.405864      0.054876     -0.054749     -0.037525   
3 -0.101699  0.824884    -0.405864      0.054876     -0.054749     -0.037525   
4 -0.130435 -1.263171    -0.405864      0.054876     -0.054749     -0.037525   
5 -0.132488  0.824884    -0.405864      0.054876     -0.054749     -0.037525   
6  0.323191  0.824884    -0.405864      0.054876     -0.054749     -0.037525   
7 -0.126330 -1.263171    -0.405864      0.054876     -0.054749     -0.037525   
8 -0.058594  0.824884    -0.405864      0.054876     -0.054749     -0.037525   
9 -0.116067  0.824884    -0.405864      0.054876     -0.054749     -0.037525   

   tcp_win_si2  
0     0.672515  
1    -1.529818  
2     0.672515  
3     0.672515  
4    -1.

c:\users\usuario\pycharmprojects\stallingevents\venv\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Counter({0: 302182, 1: 5667})
RUS METRICS(ADABOOST)
Accuracy: 98.16%
Precision: 0.00%
Recall: 0.00%
F1 Score: 0.00%
CONFUSION MATRIX:
[[302285      0]
 [  5667      0]]

RUS METRICS(RF)
dep: 1
Accuracy: 98.16%
Precision: 0.00%
Recall: 0.00%
F1 Score: 0.00%
CONFUSION MATRIX:
[[302285      0]
 [  5667      0]]

dep: 2
Accuracy: 98.16%
Precision: 0.00%
Recall: 0.00%
F1 Score: 0.00%
CONFUSION MATRIX:
[[302285      0]
 [  5667      0]]

dep: 3
Accuracy: 98.16%
Precision: 0.00%
Recall: 0.00%
F1 Score: 0.00%
CONFUSION MATRIX:
[[302285      0]
 [  5667      0]]

dep: 4
Accuracy: 98.16%
Precision: 0.00%
Recall: 0.00%
F1 Score: 0.00%
CONFUSION MATRIX:
[[302285      0]
 [  5667      0]]



c:\users\usuario\pycharmprojects\stallingevents\venv\lib\site-packages\sklearn\metrics\_classification.py:1268: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\usuario\pycharmprojects\stallingevents\venv\lib\site-packages\sklearn\metrics\_classification.py:1268: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\usuario\pycharmprojects\stallingevents\venv\lib\site-packages\sklearn\metrics\_classification.py:1268: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\usuario\pycharmprojects\s